In [55]:
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST
import os
import torch.nn.functional as F
import cv2
import numpy as np
import torch.utils.data as Data
from PIL import  Image,ImageDraw
import numpy as np
import random

Batch_Size = 50
num_epochs = 200
learning_rate = 1e-5
samples = 10000

data1 = []
data2= []


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% --  Generate dataset

im1 = Image.open('tree1.png')
im1 = im1.resize((10,10))

im2 = Image.open('tree2.png')
im2 = im2.resize((10,10))

im3 = Image.open('tree3.png')
im3 = im3.resize((10,10))

im_drone = Image.open('drone4-1.png')
im_drone = im_drone.resize((20,20))

for i in range(samples):
    
    switch = random.randint(0,1)
    if switch==0:
        img = Image.new('RGB', (100, 100), color = '#9b7653')
        img.paste(im1,(5,75),im1)
        img.paste(im1,(10,55),im1)
        img.paste(im1,(25,50),im1)
        img.paste(im2,(75,5),im2)
        img.paste(im2,(60,30),im2)
        img.paste(im2,(90,20),im2)
        img.paste(im3,(75,60),im3)
        img.paste(im3,(60,80),im3)
        
        img_b = Image.new('RGB', (100, 100), color = '#9b7653')
        img_b.paste(im1,(5,75),im1)
        img_b.paste(im1,(10,55),im1)
        img_b.paste(im1,(25,50),im1)
        img_b.paste(im2,(75,5),im2)
        img_b.paste(im2,(60,30),im2)
        img_b.paste(im2,(90,20),im2)
        img_b.paste(im3,(75,60),im3)
        img_b.paste(im3,(60,80),im3)
        
        x = random.randint(0,1)
        if x==0:
            img.paste(im_drone,(10,10),im_drone)
            data1.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
            img_b.paste(im_drone,(80,80),im_drone)
            data2.append(np.array(np.transpose(np.asarray(img_b),(2,0,1)),dtype=np.float32))
        elif x==1:
            img.paste(im_drone,(50,50),im_drone)
            data1.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
            img_b.paste(im_drone,(80,80),im_drone)
            data2.append(np.array(np.transpose(np.asarray(img_b),(2,0,1)),dtype=np.float32))
        #mg.show()
        
    elif switch==1:
        img_blank = Image.new('RGB', (100, 100), color = '#9b7653')
        img_blank.paste(im1,(5,75),im1)
        img_blank.paste(im1,(10,55),im1)
        img_blank.paste(im1,(25,50),im1)
        img_blank.paste(im2,(75,5),im2)
        img_blank.paste(im2,(60,30),im2)
        img_blank.paste(im2,(90,20),im2)
        img_blank.paste(im3,(75,60),im3)
        img_blank.paste(im3,(60,80),im3)
        #mg_blank.show()
        data1.append(np.array(np.transpose(np.asarray(img_blank),(2,0,1)),dtype=np.float32))
        data2.append(np.array(np.transpose(np.asarray(img_blank),(2,0,1)),dtype=np.float32))


#%%%%%%%%%%%%%%%%%%%%%%%%%%% Noisy data
img = Image.new('RGB', (100,100), color = '#9b7653')
img.paste(im1,(5,75),im1)
img.paste(im1,(10,55),im1)
img.paste(im1,(25,50),im1)
img.paste(im2,(75,5),im2)
img.paste(im2,(60,30),im2)
img.paste(im2,(90,20),im2)
img.paste(im3,(75,60),im3)
img.paste(im3,(60,80),im3)
im_n = np.reshape(img,(100,100,3))
b = np.asarray(im_n)
b = np.array(b, dtype=np.float32)
b = np.transpose(b,(2,0,1))
b = torch.from_numpy(b)

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%       

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Data Preparation

y0 = np.asarray(data1)
y1 = np.asarray(data2)



print(np.shape(y0))

x = torch.from_numpy(y0)
y = torch.from_numpy(y1)


torch_dataset = Data.TensorDataset(x,y)


loader = Data.DataLoader(

    dataset=torch_dataset,

    batch_size=Batch_Size,

    shuffle=True,

    num_workers=0,

)


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Defining NN

class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()

        # Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()

        # Max pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)

        # Convolution 2
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.relu2 = nn.ReLU()

        # Max pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)

        # Fully connected 1 (readout)
        self.fc1 = nn.Linear(20000, 1)
        self.sigmoid1 = nn.Sigmoid()

    def forward(self, x):
        # Convolution 1
        out = self.cnn1(x)
        out = self.relu1(out)

        # Max pool 1
        out = self.maxpool1(out)

        # Convolution 2 
        out = self.cnn2(out)
        out = self.relu2(out)

        # Max pool 2 
        out = self.maxpool2(out)

        # Resize
        # Original size: (100, 32, 7, 7)
        # out.size(0): 100
        # New out size: (100, 32*7*7)
        out = out.view(out.size(0), -1)

        # Linear function (readout)
        out = self.fc1(out)
        out = self.sigmoid1(out)
        return out
    
model = CNNModel()


optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Loss Function

def h_score(fx, gy):
 
    fx = fx - fx.mean(0)

    gy = gy - gy.mean(0)

    Nsamples = fx.size(0)
    #print(fx, np.shape(fx))
    #print(gy, np.shape(gy))
    #print(np.shape((torch.transpose(fx,3,2))),np.shape(fx))
    covf = torch.matmul(fx.t(), fx) / Nsamples

    covg = torch.matmul(gy.t(), gy) / Nsamples

    h = -2 * torch.mean((fx * gy).sum(1)) + (covf * covg).sum()

    return h


optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=1e-4)





for epoch in range(num_epochs):
    for x,y in loader:
        optimizer.zero_grad()
        # ===================forward=====================
        #loss = criterion(output1, img)
        loss = h_score(model(x),model(y))
        # ===================backward====================

        #optimizer_1.zero_grad()
        loss.backward()
        optimizer.step()
        #optimizer_1.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'.format(epoch+1, num_epochs, loss.item()))
    print("x: ", model(x[0].reshape(1,3,100, 100)))
    print("y:", model(y[0].reshape(1,3,100, 100)))
    print("b:", model(b.reshape(1,3,100, 100)))


(10000, 3, 100, 100)
epoch [1/200], loss:0.0000
x:  tensor([[1.]], grad_fn=<SigmoidBackward>)
y: tensor([[1.]], grad_fn=<SigmoidBackward>)
b: tensor([[1.]], grad_fn=<SigmoidBackward>)
epoch [2/200], loss:-0.0000
x:  tensor([[1.0000]], grad_fn=<SigmoidBackward>)
y: tensor([[1.0000]], grad_fn=<SigmoidBackward>)
b: tensor([[1.0000]], grad_fn=<SigmoidBackward>)
epoch [3/200], loss:0.0000
x:  tensor([[1.]], grad_fn=<SigmoidBackward>)
y: tensor([[1.]], grad_fn=<SigmoidBackward>)
b: tensor([[1.]], grad_fn=<SigmoidBackward>)
epoch [4/200], loss:0.0000
x:  tensor([[1.]], grad_fn=<SigmoidBackward>)
y: tensor([[1.]], grad_fn=<SigmoidBackward>)
b: tensor([[1.]], grad_fn=<SigmoidBackward>)
epoch [5/200], loss:0.0000
x:  tensor([[1.]], grad_fn=<SigmoidBackward>)
y: tensor([[1.]], grad_fn=<SigmoidBackward>)
b: tensor([[1.]], grad_fn=<SigmoidBackward>)
epoch [6/200], loss:0.0000
x:  tensor([[1.]], grad_fn=<SigmoidBackward>)
y: tensor([[1.]], grad_fn=<SigmoidBackward>)
b: tensor([[1.]], grad_fn=<Sigmoi

epoch [47/200], loss:-0.4321
x:  tensor([[1.0000]], grad_fn=<SigmoidBackward>)
y: tensor([[1.0000]], grad_fn=<SigmoidBackward>)
b: tensor([[6.1600e-07]], grad_fn=<SigmoidBackward>)
epoch [48/200], loss:-0.4279
x:  tensor([[6.0566e-07]], grad_fn=<SigmoidBackward>)
y: tensor([[6.0566e-07]], grad_fn=<SigmoidBackward>)
b: tensor([[6.0566e-07]], grad_fn=<SigmoidBackward>)
epoch [49/200], loss:-0.4351
x:  tensor([[1.0000]], grad_fn=<SigmoidBackward>)
y: tensor([[1.0000]], grad_fn=<SigmoidBackward>)
b: tensor([[6.0672e-07]], grad_fn=<SigmoidBackward>)
epoch [50/200], loss:-0.4077
x:  tensor([[1.0000]], grad_fn=<SigmoidBackward>)
y: tensor([[1.0000]], grad_fn=<SigmoidBackward>)
b: tensor([[5.9359e-07]], grad_fn=<SigmoidBackward>)
epoch [51/200], loss:-0.3878
x:  tensor([[5.8904e-07]], grad_fn=<SigmoidBackward>)
y: tensor([[5.8904e-07]], grad_fn=<SigmoidBackward>)
b: tensor([[5.8904e-07]], grad_fn=<SigmoidBackward>)
epoch [52/200], loss:-0.4375
x:  tensor([[1.0000]], grad_fn=<SigmoidBackward>)


epoch [92/200], loss:-0.4157
x:  tensor([[1.0000]], grad_fn=<SigmoidBackward>)
y: tensor([[1.0000]], grad_fn=<SigmoidBackward>)
b: tensor([[5.3801e-07]], grad_fn=<SigmoidBackward>)
epoch [93/200], loss:-0.4321
x:  tensor([[1.0000]], grad_fn=<SigmoidBackward>)
y: tensor([[1.0000]], grad_fn=<SigmoidBackward>)
b: tensor([[5.5343e-07]], grad_fn=<SigmoidBackward>)
epoch [94/200], loss:-0.4224
x:  tensor([[1.0000]], grad_fn=<SigmoidBackward>)
y: tensor([[1.0000]], grad_fn=<SigmoidBackward>)
b: tensor([[5.6215e-07]], grad_fn=<SigmoidBackward>)
epoch [95/200], loss:-0.4351
x:  tensor([[1.0000]], grad_fn=<SigmoidBackward>)
y: tensor([[1.0000]], grad_fn=<SigmoidBackward>)
b: tensor([[5.5124e-07]], grad_fn=<SigmoidBackward>)
epoch [96/200], loss:-0.4369
x:  tensor([[1.0000]], grad_fn=<SigmoidBackward>)
y: tensor([[1.0000]], grad_fn=<SigmoidBackward>)
b: tensor([[5.7021e-07]], grad_fn=<SigmoidBackward>)
epoch [97/200], loss:-0.4369
x:  tensor([[6.0287e-07]], grad_fn=<SigmoidBackward>)
y: tensor([[

epoch [137/200], loss:-0.4375
x:  tensor([[6.3961e-07]], grad_fn=<SigmoidBackward>)
y: tensor([[6.3961e-07]], grad_fn=<SigmoidBackward>)
b: tensor([[6.3961e-07]], grad_fn=<SigmoidBackward>)
epoch [138/200], loss:-0.4375
x:  tensor([[6.6022e-07]], grad_fn=<SigmoidBackward>)
y: tensor([[6.6022e-07]], grad_fn=<SigmoidBackward>)
b: tensor([[6.6022e-07]], grad_fn=<SigmoidBackward>)
epoch [139/200], loss:-0.4369
x:  tensor([[6.9991e-07]], grad_fn=<SigmoidBackward>)
y: tensor([[6.9991e-07]], grad_fn=<SigmoidBackward>)
b: tensor([[6.9991e-07]], grad_fn=<SigmoidBackward>)
epoch [140/200], loss:-0.4351
x:  tensor([[6.6672e-07]], grad_fn=<SigmoidBackward>)
y: tensor([[6.6672e-07]], grad_fn=<SigmoidBackward>)
b: tensor([[6.6672e-07]], grad_fn=<SigmoidBackward>)
epoch [141/200], loss:-0.4351
x:  tensor([[1.0000]], grad_fn=<SigmoidBackward>)
y: tensor([[1.0000]], grad_fn=<SigmoidBackward>)
b: tensor([[5.8553e-07]], grad_fn=<SigmoidBackward>)
epoch [142/200], loss:-0.4351
x:  tensor([[1.0000]], grad_

epoch [181/200], loss:-0.4224
x:  tensor([[1.0000]], grad_fn=<SigmoidBackward>)
y: tensor([[1.0000]], grad_fn=<SigmoidBackward>)
b: tensor([[5.6623e-07]], grad_fn=<SigmoidBackward>)
epoch [182/200], loss:-0.4375
x:  tensor([[5.9078e-07]], grad_fn=<SigmoidBackward>)
y: tensor([[5.9078e-07]], grad_fn=<SigmoidBackward>)
b: tensor([[5.9078e-07]], grad_fn=<SigmoidBackward>)
epoch [183/200], loss:-0.4321
x:  tensor([[1.0000]], grad_fn=<SigmoidBackward>)
y: tensor([[1.0000]], grad_fn=<SigmoidBackward>)
b: tensor([[6.0628e-07]], grad_fn=<SigmoidBackward>)
epoch [184/200], loss:-0.4321
x:  tensor([[6.2403e-07]], grad_fn=<SigmoidBackward>)
y: tensor([[6.2403e-07]], grad_fn=<SigmoidBackward>)
b: tensor([[6.2403e-07]], grad_fn=<SigmoidBackward>)
epoch [185/200], loss:-0.4321
x:  tensor([[5.7445e-07]], grad_fn=<SigmoidBackward>)
y: tensor([[5.7445e-07]], grad_fn=<SigmoidBackward>)
b: tensor([[5.7445e-07]], grad_fn=<SigmoidBackward>)
epoch [186/200], loss:-0.4369
x:  tensor([[1.0000]], grad_fn=<Sigm

In [70]:
data3=[]

for i in range(1):
    img = Image.new('RGB', (100, 100), color = '#9b7653')
    img.paste(im1,(5,75),im1)
    img.paste(im1,(10,55),im1)
    img.paste(im1,(25,50),im1)
    img.paste(im2,(75,5),im2)
    img.paste(im2,(60,30),im2)
    img.paste(im2,(90,20),im2)
    img.paste(im3,(75,60),im3)
    img.paste(im3,(60,80),im3)
    img.paste(im_drone,(50,50),im_drone)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(50,50),im_drone)
    img.show()
    #im2_pos1 = np.reshape(im2_pos1,(100,100,3))

    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    print("x: ", model(x_1[i].reshape(1,3,100,100)))

x:  tensor([[1.0000]], grad_fn=<SigmoidBackward>)


In [18]:
import random
x = random.randint(0,2)
print(x)

0
